In [ ]:
import torch
from diffusers import StableDiffusionPipeline

BASE_MODEL = "runwayml/stable-diffusion-v1-5"
LORA_DIR   = "/kaggle/input/fine-tuning-stable-diffusion-5000iterations"  # 里面有 pytorch_lora_weights.safetensors
OUT_IMG    = "/kaggle/working/sample_lolicon.png"

pipe = StableDiffusionPipeline.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    safety_checker=None,          # 关掉 NSFW 审查
    feature_extractor=None        # diffusers<=0.34 关安全检查时建议一起设 None
).to("cuda")

# 加载 LoRA（会自动找 lora_out 下的权重）
pipe.load_lora_weights(LORA_DIR)
# 如果你之后会频繁推理，可融合： pipe.fuse_lora()

# 一些省显存/稳妥设置（可选）
pipe.enable_attention_slicing()
pipe.enable_vae_tiling()

In [ ]:
# prompt = ["a <lolicon> League of Legends style icon of a magical girl with pink hair and star-shaped eyes, wearing a shining tiara and celestial cloak",
#           "a <lolicon> League of Legends style icon of a chibi-style mascot character, fluffy hair, oversized eyes, playful smile, colorful background",
#           "a <lolicon> League of Legends style icon of a little dog",
#           "a <lolicon> League of Legends style icon of a mighty elephant warrior with glowing blue armor, golden tusks, magical runes on its skin, epic fantasy lighting, cartoon style",
#           "a <lolicon> League of Legends style icon of a cute chibi elephant with oversized ears, playful smile, colorful jewels, magical aura, cartoon character",
#           "a <lolicon> League of Legends style icon of a League of Legends style icon of a minimal cartoon elephant, outlined style, flat colors, no background",
#           "a <lolicon> League of Legends style icon of a cartoon girl with short black hair and glasses, clean simple background",
#           "a <lolicon> League of Legends style icon of a cartoon character with a red nose and a nosey nose",
#           "a League of Legends style icon of a mystical mage with glowing staff"]

prompt = [
    "a <lolicon> League of Legends style icon of a mystical mage with glowing staff",
    "a <lolicon> League of Legends style icon of a dark knight in heavy armor",
    "a <lolicon> League of Legends style icon of a radiant archer with golden bow",
    "a <lolicon> League of Legends style icon of a cartoon magical girl with short black hair and glasses, flat colors, simple background",
    "a <lolicon> League of Legends style icon of a celestial sorceress surrounded by stars"
]


negative_prompt = "realistic, photo, nude, nsfw, text, watermark, logo"

generator = torch.Generator(device="cuda").manual_seed(43)
images = pipe(
    prompt[3],
    negative_prompt=negative_prompt,
    num_inference_steps=40,
    guidance_scale=4,
    generator=generator,
    num_images_per_prompt=4,
    height = 512,
    width = 512
).images


In [ ]:
from IPython.display import display
for img in images:
    display(img)